<a href="https://colab.research.google.com/github/emichester/03MIAR_Algoritmos_de_Optimizacion/blob/main/AG3/Algoritmo_Genetico_para_el_TSP_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instalación de librerias

In [3]:
!pip install requests
import urllib.request

!pip install tsplib95
import tsplib95

In [4]:
import random   #Libreria para generar numeros y listas aleatorias
import copy     #Permite hacer copias de objetos en python: listas, diccionarios,...
import numpy as np

#Carga de datos del problema

In [5]:
#Librerias y carga del problema

#http://elib.zib.de/pub/mp-testdata/tsp/tsplib/
#Documentacion :
  # https://wwwproxy.iwr.uni-heidelberg.de/groups/comopt/software/TSPLIB95/tsp95.pdf
  # https://tsplib95.readthedocs.io/usage.html
  # https://tsplib95.readthedocs.io/modules.html#module-tsplib95.models

#Matriz de distancias
file = "swiss42.tsp" ;
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos

#Objeto de tsplib95 para nuestro problema problema
problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())
print(Aristas)

#Coordenadas(si estan disponibles en el ficher)
problem.get_display(1)

#Distancia
problem.get_weight(1, 2)



gzip: swiss42.tsp already exists; do you wish to overwrite (y or n)? ^C
[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (0, 15), (0, 16), (0, 17), (0, 18), (0, 19), (0, 20), (0, 21), (0, 22), (0, 23), (0, 24), (0, 25), (0, 26), (0, 27), (0, 28), (0, 29), (0, 30), (0, 31), (0, 32), (0, 33), (0, 34), (0, 35), (0, 36), (0, 37), (0, 38), (0, 39), (0, 40), (0, 41), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (1, 21), (1, 22), (1, 23), (1, 24), (1, 25), (1, 26), (1, 27), (1, 28), (1, 29), (1, 30), (1, 31), (1, 32), (1, 33), (1, 34), (1, 35), (1, 36), (1, 37), (1, 38), (1, 39), (1, 40), (1, 41), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, 15), (2, 16), (2, 17), (2, 18), (2, 19), (2, 20), (2, 21), (2,

34

#Funciones de la Actividad Guiada 3

In [16]:
#Funciones de la Actividad Guiada 3
#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos):
  solucion = [Nodos[0]]
  for n in Nodos[1:] :
    solucion = solucion + [random.choice(list(set(Nodos) - set(solucion)))]
  return solucion

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)


#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

#Funciones Auxiliares

In [17]:
#Genera una poblacion inicial de soluciones de tamaño N.
# Puede ser válida la solución aleatoria de la AG3: crear_solucion(Nodos)
def generar_poblacion(Nodos,N):
  poblacion = []
  for i in range(N):
    poblacion+= [crear_solucion(Nodos)]
  return poblacion

In [18]:
#Evalua la población y devuelve el mejor individuo
def Evaluar_Poblacion(poblacion, problem):
  distancias= []
  for sol in poblacion:
    distancias+= [distancia_total(sol, problem)]
  return poblacion[distancias.index(min(distancias))], min(distancias)

In [19]:
#Funcion de cruce. Recibe una poblacion(lista de soluciones) y devuelve la población ampliada con los hijos.
# Todos los individuos de la población son selecionados para el cruce(si la población es par)
# Podría aplicarse un proceso previo de selección para elegir los individuos que se desea cruzar.
def Cruzar(poblacion, mutacion, problem):
  len_poblacion= len(poblacion)
  poblacion_modified= poblacion.copy()
  random.shuffle(poblacion_modified)
  while poblacion_modified:
    padre_1, padre_2= poblacion_modified.pop(), poblacion_modified.pop()
    hijo_1, hijo_2= Descendencia((padre_1, padre_2), problem, mutacion)
    poblacion.append(hijo_1)
    poblacion.append(hijo_2)
  return poblacion

In [20]:
#Funcion para generar hijos a partir de 2 padres:
# Se elige el metodo de 1-punto de corte pero es posible usar otros n-puntos, uniforme, dependiendo del problema
def Descendencia(padres, problem, mutacion):
  padre_1, padre_2= padres
  # cambia para cada corte para tener soluciones variadas
  n_split= random.randint(0, len(padre_1) - 1)
  # print(n_split)
  # print("padre_1", len(padre_1))
  # print("padre_2", len(padre_2))
  hijo_1, hijo_2= padre_1[:n_split] + padre_2[n_split:], padre_2[:n_split] + padre_1[n_split:]
  # print("hijo_1", len(hijo_1))
  # print("hijo_2", len(hijo_2))
  hijo_1, hijo_2= Mutar(Factibilizar(hijo_1, problem)), Mutar(Factibilizar(hijo_2, problem))

  return hijo_1, hijo_2

In [21]:
if not set([1,2,3,4,5]) - set([1,3,4]):
  print("Factible")

In [28]:
#Para el operador de cruce 1-punto los hijos generados no son soluciones(algunos nodos se repiten y otros no están)
def Factibilizar(solucion,problem):
  # print("Solución: ",solucion)
  if not set(Nodos) ^ set(solucion):
    # print("Factible???")
    # print("Complemento a solución: ",set(Nodos) - set(solucion))
    return solucion
  else:
    # print("No Factible")
    n_puntos_restantes= list(set(Nodos) ^ set(solucion))
    for i in range(len(solucion)):
      if solucion.count(solucion[i]) > 1:
        solucion[i] = n_puntos_restantes.pop(0)
        # print(solucion)
  # print("Fin")
  # exit()
  return solucion

In [23]:
#Funcion de mutación. Se eligen dos nodos y se intercambia. Se podrian añadir otros operaradores
# Se hace mutaciones mutacion% de las veces
def Mutar(solucion):
  indice_1, indice_2= random.randint(0, len(solucion) - 1), random.randint(0, len(solucion) - 1)
  solucion[indice_1], solucion[indice_2]= solucion[indice_2], solucion[indice_1]
  return solucion

In [24]:
#Funcion de seleccion de la población. Recibe como parametro una poblacion y
# devuelve una poblacion a la que se ha eliminado individuos poco aptos(fitness alto) y para mantener una poblacion estable de N individuos
#Se tiene en cuenta el porcentaje elitismo pasado como parametro
# Para los individuos que no son de la elite podríamos usar una selección de ruleta(proporcional a su fitness)
def Seleccionar(problem,poblacion, N, elitismo):
  if len(poblacion) >= N:
    distancia= []
    for solucion in poblacion:
      distancia+= [distancia_total(solucion, problem)]
    distancia, poblacion= np.array(distancia), np.array(poblacion)
    poblacion= poblacion[np.argsort(distancia)]
    poblacion= poblacion[:N]
    return poblacion.tolist()
  else:
    return poblacion

#Proceso Principal

In [29]:
#Funcion principal del algoritmo genetico
#######################################################3
def algoritmo_genetico(problem=problem,N=100,mutacion=.15,elitismo=.1,generaciones=100):
  # problem = datos del problema
  # N = Tamaño de la población
  # mutacion = probabilidad de una mutación
  # elitismo = porcion de la mejor poblacion a mantener
  # generaciones = nº de generaciones a generar para finalizar

  #Genera la poblacion inicial
  Nodos = list(problem.get_nodes())
  poblacion = generar_poblacion(Nodos,N)

  #Inicializamos valores para la mejor solucion
  (mejor_solucion, mejor_distancia) = Evaluar_Poblacion(poblacion, problem)


  #Condicion de parada
  parar = False
  n=0
  #Inciamos el cliclo de generaciones
  while(parar == False) :

    #Cruce de la poblacion(incluye mutación)
    poblacion = Cruzar(poblacion, mutacion, problem)

    #Seleccionamos la población
    poblacion = Seleccionar(problem,poblacion, N, elitismo)

    #Evaluamos la nueva población
    (mejor_solucion, mejor_distancia) = Evaluar_Poblacion(poblacion, problem)

    print("Generacion #", n, "\nLa mejor solución es:" , mejor_solucion, "\ncon distancia " , mejor_distancia, "\n")

    #Numero de generaciones. Criterio de parada
    if n==generaciones:
      parar = True
    n +=1

  return mejor_solucion


sol = algoritmo_genetico(problem=problem,N=500,mutacion=.3,elitismo=.40,generaciones=250)
print(f"Solución: {sol}")
print(f"Distancia: {distancia_total(sol, problem)}")

Generacion # 0 
La mejor solución es: [0, 12, 31, 20, 36, 41, 18, 29, 9, 30, 1, 27, 5, 4, 7, 13, 14, 19, 15, 26, 16, 17, 25, 21, 32, 33, 6, 28, 22, 3, 35, 37, 34, 11, 39, 10, 38, 24, 40, 23, 8, 2] 
con distancia  3846 

Generacion # 1 
La mejor solución es: [0, 2, 3, 8, 1, 20, 4, 7, 17, 14, 15, 19, 27, 5, 29, 33, 35, 36, 30, 37, 26, 24, 9, 40, 38, 12, 41, 6, 16, 28, 23, 39, 21, 11, 25, 22, 18, 10, 31, 34, 32, 13] 
con distancia  3739 

Generacion # 2 
La mejor solución es: [0, 2, 3, 8, 1, 20, 4, 7, 17, 14, 15, 19, 27, 5, 29, 33, 35, 36, 30, 37, 26, 24, 9, 40, 38, 12, 41, 6, 16, 28, 23, 39, 21, 11, 25, 22, 18, 10, 31, 34, 32, 13] 
con distancia  3739 

Generacion # 3 
La mejor solución es: [0, 1, 10, 27, 39, 11, 17, 15, 16, 13, 18, 9, 21, 12, 29, 30, 3, 40, 22, 24, 26, 35, 36, 33, 32, 31, 37, 41, 7, 4, 14, 19, 38, 6, 5, 8, 23, 25, 20, 34, 28, 2] 
con distancia  3731 

Generacion # 4 
La mejor solución es: [0, 2, 23, 28, 8, 29, 26, 30, 32, 34, 37, 11, 25, 4, 24, 39, 40, 9, 21, 6, 19, 41,